In [238]:
%%writefile ../models/train.py

import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
import fire

# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

CLASS_NAMES = ['10', '100', '20', '200', '5', '50', '500']
train_data_path = "gs://team4-cash-classifyimgs/euro/train_set.csv"
eval_data_path = "gs://team4-cash-classify/imgs/euro/eval_set.csv"

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3


def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

# Call model.predict() on a few images in the evaluation dataset
def plot_predictions(model, filename):
  f, ax = plt.subplots(7, 5, figsize=(25,15))
  dataset = (tf.data.TextLineDataset(filename).
      map(decode_csv))
  for idx, (img, label) in enumerate(dataset.take(35)):
    ax[idx//5, idx%5].imshow((img.numpy()));
    batch_image = tf.reshape(img, [1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    batch_pred = model.predict(batch_image)
    pred = batch_pred[0]
    label = CLASS_NAMES[label.numpy()]
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASS_NAMES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx//5, idx%5].set_title('{}: {} ({:.4f})'.format(label, pred_label, prob))
    ax[idx//5, idx%5].axis('off')

def show_trained_weights(model):
  # CLASS_NAMES is ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
  LAYER = 1 # Layer 0 flattens the image, layer=1 is the first dense layer
  WEIGHT_TYPE = 0 # 0 for weight, 1 for bias

  f, ax = plt.subplots(1, 5, figsize=(15,15))
  for flower in range(len(CLASS_NAMES)):
    weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
    min_wt = tf.math.reduce_min(weights).numpy()
    max_wt = tf.math.reduce_max(weights).numpy()
    flower_name = CLASS_NAMES[flower]
    print("Scaling weights for {} in {} to {}".format(
        flower_name, min_wt, max_wt))
    weights = (weights - min_wt)/(max_wt - min_wt)
    ax[flower].imshow(weights.reshape(IMG_HEIGHT, IMG_WIDTH, 3));
    ax[flower].set_title(flower_name);a
    ax[flower].axis('off')


def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)



# the label is the index into CLASS_NAMES array
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

# parameterize to the values in the previous cell
def train_and_evaluate(train_data_path=f'gs://team4-cash-classify/imgs/euro/train_set.csv',
                       eval_data_path=f'gs://team4-cash-classify/imgs/euro/eval_set.csv',
                       CLASS_NAMES = ['10', '100', '20' ,'200', '5', '50', '500'], 
                       batch_size = 32,
                       lrate = 0.001,
                       l1 = 0.,
                       l2 = 0.,
                       num_hidden = 16):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  train_dataset = (tf.data.TextLineDataset(
      train_data_path).
      map(decode_csv)).batch(batch_size)

  eval_dataset = (tf.data.TextLineDataset(
      eval_data_path).
      map(decode_csv)).batch(32) # this doesn't matter

  layers = [
     
      tf.keras.layers.RandomFlip(),
      tf.keras.layers.RandomRotation(0.1),
      hub.KerasLayer(
          "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
          input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
          trainable=False,
          name='mobilenet_embedding'),

      tf.keras.layers.Dense(num_hidden,
                            kernel_regularizer=regularizer, 
                            activation='relu',
                            name='dense_hidden'),
      tf.keras.layers.Dense(len(CLASS_NAMES), 
                            kernel_regularizer=regularizer,
                            activation='softmax',
                            name='flower_prob')
  ]

  model = tf.keras.Sequential(layers, name='flower_classification')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=False),
                metrics=['accuracy'])
  
  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)    
  # Save the model    
  #model_filename = "model.pkl"
  #with open(model_filename, "wb") as model_file:
  #    pickle.dump(history, model_file)
  #gcs_model_path = f"gs://team4-cash-classify/{model_filename}"
  #subprocess.check_call(
  #   ["gsutil", "cp", model_filename, gcs_model_path], stderr=sys.stdout
  #)
  gcs_model_path = f"gs://team4-cash-classify/cash-classify-model-v1"  
  #model.save(os.getenv('AIP_MODEL_DIR'))
  model.save(gcs_model_path)
  
  

if __name__ == "__main__":
    fire.Fire(train_and_evaluate)



Overwriting ../models/train.py


In [266]:
! python ../models/train.py

2022-04-27 17:46:50.574122: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Epoch 1/10
20/20 [==============================] - 60s 3s/step - loss: 1.5268 - accuracy: 0.4437 - val_loss: 1.3453 - val_accuracy: 0.5000
Epoch 2/10
20/20 [==============================] - 38s 2s/step - loss: 1.1507 - accuracy: 0.5641 - val_loss: 1.0321 - val_accuracy: 0.6062
Epoch 3/10
20/20 [==============================] - 36s 2s/step - loss: 0.8787 - accuracy: 0.6875 - val_loss: 0.8484 - val_accuracy: 0.6687
Epoch 4/10
20/20 [==============================] - 36s 2s/step - loss: 0.7154 - accuracy: 0.7406 - val_loss: 0.7464 - val_accuracy: 0.7500
Epoch 5/10
20/20 [==============================] - 40s 2s/step - loss: 0.5961 - accuracy: 0.7953 - val_loss: 0.6766 - val_accuracy: 0.8125
Epoch 6/10
20/20 [==============================] - 64s 3s/step - loss: 0.5409 - accuracy: 0.8109

In [240]:
#Pipeline
from google.cloud import aiplatform
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
print (PROJECT_ID)

qwiklabs-gcp-00-f30184915a55


In [241]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [242]:
%%writefile ../models/Dockerfile
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2 fire
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting ../models/Dockerfile


In [243]:
IMAGE_NAME = "cash_classification_vertex"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI

'gcr.io/qwiklabs-gcp-00-f30184915a55/cash_classification_vertex:latest'

In [244]:
!gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI ../models

Creating temporary tarball archive of 7 file(s) totalling 17.5 KiB before compression.
Uploading tarball of [../models] to [gs://qwiklabs-gcp-00-f30184915a55_cloudbuild/source/1651077807.043567-19d0bee933af4274a0499dc332fafee2.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-f30184915a55/locations/global/builds/85b7c0ca-b4f4-4273-87db-62aa72020b20].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/85b7c0ca-b4f4-4273-87db-62aa72020b20?project=215824651958].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "85b7c0ca-b4f4-4273-87db-62aa72020b20"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-f30184915a55_cloudbuild/source/1651077807.043567-19d0bee933af4274a0499dc332fafee2.tgz#1651077807372967
Copying gs://qwiklabs-gcp-00-f30184915a55_cloudbuild/source/1651077807.043567-19d0bee933af4274a0499dc332fafee2.tgz#1651077807372967...
/ [1 files][  3.5 KiB/  3.5 KiB]                      

In [225]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest"
)

In [226]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}


gs://qwiklabs-gcp-00-f30184915a55-kfp-artifact-store/


In [246]:
ARTIFACT_STORE = f"gs://team4-cash-classify"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/imgs"

TRAINING_FILE_PATH = f"{DATA_ROOT}/euro/train_set.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/euro/eval_set.csv"
REGION = "us-central1"
%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}

env: PIPELINE_ROOT=gs://team4-cash-classify/pipeline
env: PROJECT_ID=qwiklabs-gcp-00-f30184915a55
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-gcp-00-f30184915a55/cash_classification_vertex:latest
env: TRAINING_FILE_PATH=gs://team4-cash-classify/imgs/euro/train_set.csv
env: VALIDATION_FILE_PATH=gs://team4-cash-classify/imgs/euro/eval_set.csv


In [251]:
PIPELINE_JSON = "cash_kfp_pipeline1.json"

In [248]:
import google.cloud.aiplatform as aip
import kfp
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.dsl import component
from kfp.v2.google import experimental


In [252]:
%%writefile pipeline.py

import os

from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v2 import dsl

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

TRAINING_CONTAINER_IMAGE_URI = os.getenv("TRAINING_CONTAINER_IMAGE_URI")
SERVING_CONTAINER_IMAGE_URI = os.getenv("SERVING_CONTAINER_IMAGE_URI")
SERVING_MACHINE_TYPE = os.getenv("SERVING_MACHINE_TYPE", "n1-standard-16")

TRAINING_FILE_PATH = os.getenv("TRAINING_FILE_PATH")
VALIDATION_FILE_PATH = os.getenv("VALIDATION_FILE_PATH")

MAX_TRIAL_COUNT = int(os.getenv("MAX_TRIAL_COUNT", "5"))
PARALLEL_TRIAL_COUNT = int(os.getenv("PARALLEL_TRIAL_COUNT", "5"))
THRESHOLD = float(os.getenv("THRESHOLD", "0.6"))

PIPELINE_NAME = os.getenv("PIPELINE_NAME", "cash-classification")
BASE_OUTPUT_DIR = os.getenv("BASE_OUTPUT_DIR", PIPELINE_ROOT)
MODEL_DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME", PIPELINE_NAME)


@dsl.pipeline(
    name=f"{PIPELINE_NAME}-kfp-pipeline",
    description="Kubeflow pipeline that tunes, trains, and deploys on Vertex",
    pipeline_root=PIPELINE_ROOT,
)
def create_pipeline():

    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
                "accelerator_type": "NVIDIA_TESLA_T4",
                "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                "args": [
                    f"--train_data_path={TRAINING_FILE_PATH}",
                    f"--eval_data_path={VALIDATION_FILE_PATH}",
                ],
            },
        }
    ]

    metric_spec = hyperparameter_tuning_job.serialize_metrics(
        {"accuracy": "maximize"}
    )


    

    # Construct new worker_pool_specs and
    # train new model based on best hyperparameters

    training_task = CustomTrainingJobOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-training-job",
        worker_pool_specs= 
        [
            {
                "machine_spec": {"machine_type": "n1-standard-4"},
                "replica_count": 1,
                "container_spec": {
                    "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                    "args": [
                        f"--train_data_path={TRAINING_FILE_PATH}",
                        f"--eval_data_path={VALIDATION_FILE_PATH}",
                    ],
                },
            }
        ],
        base_output_directory=BASE_OUTPUT_DIR,
    )

    model_upload_task = ModelUploadOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-model-upload-job",
        artifact_uri=f"{BASE_OUTPUT_DIR}/model",
        serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    )
    model_upload_task.after(training_task)

    endpoint_create_task = EndpointCreateOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-create-endpoint-job",
    )
    endpoint_create_task.after(model_upload_task)

    model_deploy_op = ModelDeployOp(  # pylint: disable=unused-variable
        model=model_upload_task.outputs["model"],
        endpoint=endpoint_create_task.outputs["endpoint"],
        deployed_model_display_name=MODEL_DISPLAY_NAME,
        dedicated_resources_machine_type=SERVING_MACHINE_TYPE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )


Overwriting pipeline.py


In [250]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://team4-cash-classify/


In [253]:
from datetime import datetime

from google.cloud import aiplatform

#ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
ARTIFACT_STORE =  f"gs://team4-cash-classify"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/imgs"

TRAINING_FILE_PATH = f"{DATA_ROOT}/euro/train_set.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/euro/eval_set.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}

env: PIPELINE_ROOT=gs://team4-cash-classify/pipeline
env: PROJECT_ID=qwiklabs-gcp-00-f30184915a55
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-gcp-00-f30184915a55/cash_classification_vertex:latest
env: TRAINING_FILE_PATH=gs://team4-cash-classify/imgs/euro/train_set.csv
env: VALIDATION_FILE_PATH=gs://team4-cash-classify/imgs/euro/eval_set.csv
env: BASE_OUTPUT_DIR=gs://team4-cash-classify/models/20220427164747


In [233]:
PIPELINE_JSON = "cash_kfp_pipeline1.json"

In [40]:
!pip3 install kfp --upgrade

In [254]:
!dsl-compile-v2 --py pipeline.py --output $PIPELINE_JSON

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [255]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="cash_kfp_pipeline",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/215824651958/locations/us-central1/pipelineJobs/cash-classification-kfp-pipeline-20220427164757
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/215824651958/locations/us-central1/pipelineJobs/cash-classification-kfp-pipeline-20220427164757')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/cash-classification-kfp-pipeline-20220427164757?project=215824651958
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/215824651958/locations/us-central1/pipelineJobs/cash-classification-kfp-pipeline-20220427164757 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:Pipeli

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [custom-training-job].; Job (project_id = qwiklabs-gcp-00-f30184915a55, job_id = 7610258736535306240) is failed due to the above error.; Failed to handle the job: {project_number = 215824651958, job_id = 7610258736535306240}"


In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=create_pipeline,
    package_path="tabular regression_pipeline.json".replace(" ", "_"),
)

In [31]:
@component
def print_op(input1: str):
    print("training task: {}".format(input1))

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

try:
    if delete_model and "DISPLAY_NAME" in globals():
        models = aip.Model.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        model = models[0]
        aip.Model.delete(model)
        print("Deleted model:", model)
except Exception as e:
    print(e)

try:
    if delete_endpoint and "DISPLAY_NAME" in globals():
        endpoints = aip.Endpoint.list(
            filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
        )
        endpoint = endpoints[0]
        endpoint.undeploy_all()
        aip.Endpoint.delete(endpoint.resource_name)
        print("Deleted endpoint:", endpoint)
except Exception as e:
    print(e)

if delete_dataset and "DISPLAY_NAME" in globals():
    if "tabular" == "tabular":
        try:
            datasets = aip.TabularDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.TabularDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "image":
        try:
            datasets = aip.ImageDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.ImageDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "text":
        try:
            datasets = aip.TextDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.TextDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "video":
        try:
            datasets = aip.VideoDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.VideoDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

try:
    if delete_pipeline and "DISPLAY_NAME" in globals():
        pipelines = aip.PipelineJob.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        pipeline = pipelines[0]
        aip.PipelineJob.delete(pipeline.resource_name)
        print("Deleted pipeline:", pipeline)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME

In [262]:
hp_dict: str = '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
data_dir: str = "gs://aju-dev-demos-codelabs/bikes_weather/"
train_data_path=f'gs://team4-cash-classify/imgs/euro/train_set.csv'
eval_data_path=f'gs://team4-cash-classify/imgs/euro/eval_set.csv',
TRAINER_ARGS = []

# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/{TIMESTAMP}"

MODEL_DISPLAY_NAME = f"cash_train_deploy{TIMESTAMP}"
print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)


@kfp.dsl.pipeline(name="cash-train-endpoint-deploy" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest",
):

    train_task = print_op("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": TRAINER_ARGS,
                    "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
                    "imageUri": TRAINING_CONTAINER_IMAGE_URI,
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-16",
                    "accelerator_type": aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
                    "accelerator_count": 2,
                },
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri=serving_container_image_uri,
        # serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    gcc_aip.ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type="n1-standard-16",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

[] gs://team4-cash-classify/pipeline/20220427164747 cash_train_deploy20220427164747


In [264]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular regression_pipeline.json".replace(" ", "_"),
)

In [265]:
DISPLAY_NAME = "bikes_weather_" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular regression_pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
)

job.run()

! rm tabular_regression_pipeline.json

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/215824651958/locations/us-central1/pipelineJobs/cash-train-endpoint-deploy20220427164747-20220427170720
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/215824651958/locations/us-central1/pipelineJobs/cash-train-endpoint-deploy20220427164747-20220427170720')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/cash-train-endpoint-deploy20220427164747-20220427170720?project=215824651958
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/215824651958/locations/us-central1/pipelineJobs/cash-train-endpoint-deploy20220427164747-20220427170720 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [print-op].; Job (project_id = qwiklabs-gcp-00-f30184915a55, job_id = 4465620296723857408) is failed due to the above error.; Failed to handle the job: {project_number = 215824651958, job_id = 4465620296723857408}"


In [280]:
# Helper functions
def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

# Call model.predict() on a few images in the evaluation dataset
def plot_predictions(model, filename):
  f, ax = plt.subplots(7, 5, figsize=(25,15))
  dataset = (tf.data.TextLineDataset(filename).
      map(decode_csv))
  for idx, (img, label) in enumerate(dataset.take(35)):
    ax[idx//5, idx%5].imshow((img.numpy()));
    batch_image = tf.reshape(img, [1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    batch_pred = model.predict(batch_image)
    pred = batch_pred[0]
    label = CLASS_NAMES[label.numpy()]
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASS_NAMES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx//5, idx%5].set_title('{}: {} ({:.4f})'.format(label, pred_label, prob))
    ax[idx//5, idx%5].axis('off')

def show_trained_weights(model):
  # CLASS_NAMES is ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
  LAYER = 1 # Layer 0 flattens the image, layer=1 is the first dense layer
  WEIGHT_TYPE = 0 # 0 for weight, 1 for bias

  f, ax = plt.subplots(1, 5, figsize=(15,15))
  for flower in range(len(CLASS_NAMES)):
    weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
    min_wt = tf.math.reduce_min(weights).numpy()
    max_wt = tf.math.reduce_max(weights).numpy()
    flower_name = CLASS_NAMES[flower]
    print("Scaling weights for {} in {} to {}".format(
        flower_name, min_wt, max_wt))
    weights = (weights - min_wt)/(max_wt - min_wt)
    ax[flower].imshow(weights.reshape(IMG_HEIGHT, IMG_WIDTH, 3));
    ax[flower].set_title(flower_name);a
    ax[flower].axis('off')
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)



# the label is the index into CLASS_NAMES array
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

In [294]:
import tensorflow_hub as hub
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

CLASS_NAMES = ['10', '100', '20', '200', '5', '50', '500']
train_data_path = "gs://team4-cash-classify/imgs/euro/train_set.csv"
eval_data_path = "gs://team4-cash-classify/imgs/euro/eval_set.csv"

# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

# parameterize to the values in the previous cell
def create_model(batch_size = 32,
                       lrate = 0.001,
                       l1 = 0.,
                       l2 = 0.,
                       num_hidden = 16):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)



  # train_dataset2 = tf.keras.layers.RandomFlip()(train_dataset)
  #eval_dataset2 = tf.keras.layers.RandomFlip()(eval_dataset2)

  layers = [
     
      tf.keras.layers.RandomFlip(),
      tf.keras.layers.RandomRotation(0.1),
      hub.KerasLayer(
          "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
          input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
          trainable=False,
          name='mobilenet_embedding'),

      tf.keras.layers.Dense(num_hidden,
                            kernel_regularizer=regularizer, 
                            activation='relu',
                            name='dense_hidden'),
      tf.keras.layers.Dense(len(CLASS_NAMES), 
                            kernel_regularizer=regularizer,
                            activation='softmax',
                            name='flower_prob')
  ]

  model = tf.keras.Sequential(layers, name='flower_classification')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=False),
                metrics=['accuracy'])
  # model.build()
  # print(model.summary())
  #history = model.fit(train_dataset, validation_data=eval_dataset, epochs=2)
  #training_plot(['loss', 'accuracy'], history)
  return model

In [295]:
def train_and_evaluate(model,epochs,batch_size=32):
  train_dataset = (tf.data.TextLineDataset(
      train_data_path).
      map(decode_csv)).batch(batch_size)

  eval_dataset = (tf.data.TextLineDataset(
      eval_data_path).
      map(decode_csv)).batch(32) # this doesn't matter
  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=epochs)
  training_plot(['loss', 'accuracy'], history)
  return model

In [296]:
model=create_model()

In [ ]:
model = train_and_evaluate(model,5)

Epoch 1/5
20/20 [==============================] - 76s 3s/step - loss: 1.5351 - accuracy: 0.4125 - val_loss: 1.3638 - val_accuracy: 0.4688
Epoch 2/5
20/20 [==============================] - 45s 2s/step - loss: 1.1947 - accuracy: 0.5422 - val_loss: 1.1061 - val_accuracy: 0.5562
Epoch 3/5
20/20 [==============================] - 45s 2s/step - loss: 1.0031 - accuracy: 0.6469 - val_loss: 0.9449 - val_accuracy: 0.6438
Epoch 4/5


In [287]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
import fire

# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

CLASS_NAMES = ['10', '100', '20', '200', '5', '50', '500']
train_data_path = "gs://team4-cash-classifyimgs/euro/train_set.csv"
eval_data_path = "gs://team4-cash-classify/imgs/euro/eval_set.csv"

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3


def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

# Call model.predict() on a few images in the evaluation dataset
def plot_predictions(model, filename):
  f, ax = plt.subplots(7, 5, figsize=(25,15))
  dataset = (tf.data.TextLineDataset(filename).
      map(decode_csv))
  for idx, (img, label) in enumerate(dataset.take(35)):
    ax[idx//5, idx%5].imshow((img.numpy()));
    batch_image = tf.reshape(img, [1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    batch_pred = model.predict(batch_image)
    pred = batch_pred[0]
    label = CLASS_NAMES[label.numpy()]
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASS_NAMES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx//5, idx%5].set_title('{}: {} ({:.4f})'.format(label, pred_label, prob))
    ax[idx//5, idx%5].axis('off')

def show_trained_weights(model):
  # CLASS_NAMES is ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
  LAYER = 1 # Layer 0 flattens the image, layer=1 is the first dense layer
  WEIGHT_TYPE = 0 # 0 for weight, 1 for bias

  f, ax = plt.subplots(1, 5, figsize=(15,15))
  for flower in range(len(CLASS_NAMES)):
    weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
    min_wt = tf.math.reduce_min(weights).numpy()
    max_wt = tf.math.reduce_max(weights).numpy()
    flower_name = CLASS_NAMES[flower]
    print("Scaling weights for {} in {} to {}".format(
        flower_name, min_wt, max_wt))
    weights = (weights - min_wt)/(max_wt - min_wt)
    ax[flower].imshow(weights.reshape(IMG_HEIGHT, IMG_WIDTH, 3));
    ax[flower].set_title(flower_name);a
    ax[flower].axis('off')


def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)



# the label is the index into CLASS_NAMES array
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

# parameterize to the values in the previous cell
def train_and_evaluate(train_data_path=f'gs://team4-cash-classify/imgs/euro/train_set.csv',
                       eval_data_path=f'gs://team4-cash-classify/imgs/euro/eval_set.csv',
                       CLASS_NAMES = ['10', '100', '20' ,'200', '5', '50', '500'], 
                       batch_size = 32,
                       lrate = 0.001,
                       l1 = 0.,
                       l2 = 0.,
                       num_hidden = 16):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  train_dataset = (tf.data.TextLineDataset(
      train_data_path).
      map(decode_csv)).batch(batch_size)

  eval_dataset = (tf.data.TextLineDataset(
      eval_data_path).
      map(decode_csv)).batch(32) # this doesn't matter

  layers = [
     
      tf.keras.layers.RandomFlip(),
      tf.keras.layers.RandomRotation(0.1),
      hub.KerasLayer(
          "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
          input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
          trainable=False,
          name='mobilenet_embedding'),

      tf.keras.layers.Dense(num_hidden,
                            kernel_regularizer=regularizer, 
                            activation='relu',
                            name='dense_hidden'),
      tf.keras.layers.Dense(len(CLASS_NAMES), 
                            kernel_regularizer=regularizer,
                            activation='softmax',
                            name='flower_prob')
  ]

  model = tf.keras.Sequential(layers, name='flower_classification')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=False),
                metrics=['accuracy'])
  
  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)    
  # Save the model    
  #model_filename = "model.pkl"
  #with open(model_filename, "wb") as model_file:
  #    pickle.dump(history, model_file)
  #gcs_model_path = f"gs://team4-cash-classify/{model_filename}"
  #subprocess.check_call(
  #   ["gsutil", "cp", model_filename, gcs_model_path], stderr=sys.stdout
  #)
  gcs_model_path = f"gs://team4-cash-classify/cash-classify-model-v1"  
  #model.save(os.getenv('AIP_MODEL_DIR'))
  #model.save(gcs_model_path)
  return model




In [288]:
model=train_and_evaluate()

Epoch 1/10
20/20 [==============================] - 56s 3s/step - loss: 1.6547 - accuracy: 0.4313 - val_loss: 1.4542 - val_accuracy: 0.4563
Epoch 2/10
20/20 [==============================] - 43s 2s/step - loss: 1.3230 - accuracy: 0.5203 - val_loss: 1.2215 - val_accuracy: 0.5188
Epoch 3/10
20/20 [==============================] - 43s 2s/step - loss: 1.1575 - accuracy: 0.5891 - val_loss: 1.1020 - val_accuracy: 0.5625
Epoch 4/10
20/20 [==============================] - 42s 2s/step - loss: 1.0083 - accuracy: 0.6578 - val_loss: 0.9862 - val_accuracy: 0.6438
Epoch 5/10
20/20 [==============================] - 42s 2s/step - loss: 0.8475 - accuracy: 0.6984 - val_loss: 0.8675 - val_accuracy: 0.7000
Epoch 6/10
20/20 [==============================] - 41s 2s/step - loss: 0.7480 - accuracy: 0.7469 - val_loss: 0.8175 - val_accuracy: 0.7063
Epoch 7/10
20/20 [==============================] - 41s 2s/step - loss: 0.6839 - accuracy: 0.7484 - val_loss: 0.7573 - val_accuracy: 0.7188
Epoch 8/10
20/20 [==

In [289]:
# Save Model Iteration
model.save(f'gs://team4-cash-classify/cmodelv1')
BUCKET = "team4-cash-classify"


2022-04-27 22:04:25.782755: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://team4-cash-classify/cmodelv1/assets


In [ ]:
os.environ["EXPORT_PATH"] = f'gs://team4-cash-classify/cmodelv1'

In [293]:
%%bash
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
MODEL_DISPLAYNAME=cash_clasify_$TIMESTAMP
ENDPOINT_DISPLAYNAME=cash_clasify_endpoint_$TIMESTAMP
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest"
ARTIFACT_DIRECTORY='gs://team4-cash-classify/cmodelv1'
echo $ARTIFACT_DIRECTORY

#gsutil cp -r ${EXPORT_PATH}/* ${ARTIFACT_DIRECTORY}

# Model
MODEL_RESOURCENAME=$(gcloud ai models upload \
    --region=$REGION \
    --display-name=$MODEL_DISPLAYNAME \
    --container-image-uri=$IMAGE_URI \
    --artifact-uri=$ARTIFACT_DIRECTORY \
    --format="value(model)")

MODEL_ID=$(echo $MODEL_RESOURCENAME | cut -d"/" -f6)

echo "MODEL_DISPLAYNAME=${MODEL_DISPLAYNAME}"
echo "MODEL_RESOURCENAME=${MODEL_RESOURCENAME}"
echo "MODEL_ID=${MODEL_ID}"

# Endpoint
ENDPOINT_RESOURCENAME=$(gcloud ai endpoints create \
  --region=$REGION \
  --display-name=$ENDPOINT_DISPLAYNAME \
  --format="value(name)")

ENDPOINT_ID=$(echo $ENDPOINT_RESOURCENAME | cut -d"/" -f6)

echo "ENDPOINT_DISPLAYNAME=${ENDPOINT_DISPLAYNAME}"
echo "ENDPOINT_RESOURCENAME=${ENDPOINT_RESOURCENAME}"
echo "ENDPOINT_ID=${ENDPOINT_ID}"

# Deployment
DEPLOYEDMODEL_DISPLAYNAME=${MODEL_DISPLAYNAME}_deployment
MACHINE_TYPE=n1-standard-2
MIN_REPLICA_COUNT=1
MAX_REPLICA_COUNT=3

gcloud ai endpoints deploy-model $ENDPOINT_RESOURCENAME \
  --region=$REGION \
  --model=$MODEL_RESOURCENAME \
  --display-name=$DEPLOYEDMODEL_DISPLAYNAME \
  --machine-type=$MACHINE_TYPE \
  --min-replica-count=$MIN_REPLICA_COUNT \
  --max-replica-count=$MAX_REPLICA_COUNT \
  --traffic-split=0=100

gs://team4-cash-classify/cmodelv1
MODEL_DISPLAYNAME=cash_clasify_20220427_221643
MODEL_RESOURCENAME=projects/215824651958/locations/us-central1/models/3603394273338195968
MODEL_ID=3603394273338195968
ENDPOINT_DISPLAYNAME=cash_clasify_endpoint_20220427_221643
ENDPOINT_RESOURCENAME=projects/215824651958/locations/us-central1/endpoints/4344539877249908736
ENDPOINT_ID=4344539877249908736


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8806523607984373760]...
...................done.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [7842753287727087616]...
......done.
Created Vertex AI endpoint: projects/215824651958/locations/us-central1/endpoints/4344539877249908736.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [6225961021501079552]...
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................